In [34]:
import qiskit
import numpy as np
from qiskit.quantum_info import SparsePauliOp
from qiskit.quantum_info import SparsePauliOp

def ising(N,J,h):
    """create Hamiltonian for Ising model
        H = J * sum_{i = 0}^{N-1) Z_i*Z_{i+1} + h*sum_{i=0}^{N-1}X_i
    """

    pauli_op_list = []  
    for j in range(N):
        s = ('I' * (N - j - 1)) + 'X' + ('I' * j)
        pauli_op_list.append((s, h))
 
    for j in range(N-1):
        s = ('I' * (N - j - 2)) + 'ZZ' + ('I' * j)
        pauli_op_list.append((s, J))
        
    h_op = SparsePauliOp.from_list(pauli_op_list)

    return h_op

def XYZ_model(N,J,u,h):
    """create Hamiltonian for Ising model
        H = J * sum_{i = 0}^{N-1) Z_i*Z_{i+1} + h*sum_{i=0}^{N-1}X_i
    """

    pauli_op_list = []  
    for j in range(N):
        s = ('I' * (N - j - 1)) + 'Z' + ('I' * j)
        pauli_op_list.append((s, h[j]))
 
    for j in range(N-1):
        s = ('I' * (N - j - 2)) + 'ZZ' + ('I' * j)
        pauli_op_list.append((s, u))
    
    for j in range(N-1):
        s = ('I' * (N - j - 2)) + 'XX' + ('I' * j) + ('I' * (N - j - 2)) + 'YY' + ('I' * j)
        pauli_op_list.append((s, -J))
        
    h_op = SparsePauliOp.from_list(pauli_op_list)

    return h_op

def pauli_oper(N, oper = None):
    
    pauli_op_list = []  
    s = ('I' * (N - 1)) + oper
    pauli_op_list.append((s, 1.0))
        
    h_op = SparsePauliOp.from_list(pauli_op_list)
    return h_op    

def pauli_collect(N, oper = None):
    
    pauli_op_list = []  
    for j in range(N):
        s = ('I' * (N - j - 1)) + oper + ('I' * j)
        pauli_op_list.append((s, 1.0))
        
    h_op = SparsePauliOp.from_list(pauli_op_list)
    return h_op   

In [35]:
from scipy.linalg import expm 
def time_dependent_qc(num_qubits: int,h_opt, t):
    """create U circuit from h_opt and time t
    
    Args:
        - qc (QuantumCircuit): Init circuit
        - h_opt: Hamiltonian
        - t (float): time
        
    Returns:
        - QuantumCircuit: the added circuit
    """
    #Create circuit
    qc = qiskit.QuantumCircuit(num_qubits, num_qubits)
    
    # Ensure h_opt is Hermitian
    if not np.allclose(h_opt.to_matrix(), np.conj(h_opt.to_matrix()).T):
        raise ValueError("The Hamiltonian is not Hermitian.")

    # Calculate the unitary matrix
    U = expm(-1j * t * h_opt.to_matrix())

    # Check if U is unitary
    if not np.allclose(U @ U.conj().T, np.eye(U.shape[0])):
        raise ValueError("The resulting matrix U is not unitary.")

    from qiskit.circuit.library import UnitaryGate
    # Create a UnitaryGate from the unitary_matrix
    unitary_gate = UnitaryGate(U)

    # Append the unitary_gate to the quantum circuit
    qc.append(unitary_gate, range(qc.num_qubits))

    return qc

In [36]:
from qoop.compilation.qsp import QuantumStatePreparation
compiler = QuantumStatePreparation.prepare(U)

In [37]:
compiler.u.draw()

┌──────────────┐┌──────────────┐┌──────────────┐     ┌───┐
q_0: ┤ Rz(theta[0]) ├┤ Rx(theta[2]) ├┤ Rz(theta[4]) ├──■──┤ X ├
     ├──────────────┤├──────────────┤├──────────────┤┌─┴─┐└─┬─┘
q_1: ┤ Rz(theta[1]) ├┤ Rx(theta[3]) ├┤ Rz(theta[5]) ├┤ X ├──■──
     └──────────────┘└──────────────┘└──────────────┘└───┘

In [38]:
import qiskit
import numpy as np
from qiskit.quantum_info import SparsePauliOp
from scipy.linalg import expm  # Cần để tính toán ma trận unitary từ Hamiltonian

# Các hàm đã định nghĩa trước đó...

# Khởi tạo số qubit và Hamiltonian
num_qubits = 2  # Ví dụ: Hệ 2 qubit
J = 1.0
h = 0.5
h_opt = ising(num_qubits, J, h)

# Thời gian cần lấy trạng thái
t = 2.0

# Khởi tạo trạng thái ban đầu (|00> hoặc |0> cho mỗi qubit)
psi_0 = np.zeros(2**num_qubits, dtype=complex)
psi_0[0] = 1.0  # |00>

# Tạo toán tử tiến hóa thời gian U(t)
U = expm(-1j * t * h_opt.to_matrix())

# Kiểm tra U có unitary không
if not np.allclose(U @ U.conj().T, np.eye(U.shape[0])):
    raise ValueError("The resulting matrix U is not unitary.")

# Tính trạng thái tại thời điểm t
psi_t = U @ psi_0

# In trạng thái
print(f"Trạng thái tại thời gian t = {t}:")
print(psi_t)


Trạng thái tại thời gian t = 2.0:
[-6.83754982e-01-0.56356852j  4.16333634e-17-0.10891981j
  2.08166817e-17-0.10891981j -2.67608146e-01+0.3457289j ]
